In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd

from pyspark.sql.types import DoubleType,IntegerType, StringType
#from pyspark.sql.functions import when, udf, col, regexp_replace
import pyspark.sql.functions as f

In [ ]:
sc= SparkContext('local')
sqlCtx= SQLContext(sc)

In [ ]:
data=  [('2345', 'Checked by John'),
('2398','Verified by Stacy'),
('2328','Verified by Srinivas than some random text'),        
('3983','Double Checked on 2/23/17 by Marsha')]
df= sc.parallelize(data).toDF(['ID','Notes'] )
df.show()

+----+--------------------+
|  ID|               Notes|
+----+--------------------+
|2345|     Checked by John|
|2398|   Verified by Stacy|
|2328|Verified by Srini...|
|3983|Double Checked on...|
+----+--------------------+



### f.regexp_extract(DataFrame['컬럼'], '정규 표현식', 정규식 그룹 번호)
```
0 : 전체
1 : 첫 번째 그룹...
n : n 번째 그룹
```

In [ ]:
df.withColumn('fname', f.regexp_extract(df['Notes'], 'by [a-zA-Z]+', 0)).show()

+----+--------------------+-----------+
|  ID|               Notes|      fname|
+----+--------------------+-----------+
|2345|     Checked by John|    by John|
|2398|   Verified by Stacy|   by Stacy|
|2328|Verified by Srini...|by Srinivas|
|3983|Double Checked on...|  by Marsha|
+----+--------------------+-----------+



In [ ]:
df.withColumn('fname', f.regexp_extract(df['Notes'], '(by) ([a-zA-Z]+)', 1)).show()

+----+--------------------+-----+
|  ID|               Notes|fname|
+----+--------------------+-----+
|2345|     Checked by John|   by|
|2398|   Verified by Stacy|   by|
|2328|Verified by Srini...|   by|
|3983|Double Checked on...|   by|
+----+--------------------+-----+



In [ ]:
df.withColumn('fname', f.regexp_extract(df['Notes'], '(by) ([a-zA-Z]+)', 2)).show()

+----+--------------------+--------+
|  ID|               Notes|   fname|
+----+--------------------+--------+
|2345|     Checked by John|    John|
|2398|   Verified by Stacy|   Stacy|
|2328|Verified by Srini...|Srinivas|
|3983|Double Checked on...|  Marsha|
+----+--------------------+--------+



In [ ]:
df.withColumn('fname', f.regexp_extract(df['Notes'], '(\w+) (by) ([a-zA-Z]+)', 1)).show()

+----+--------------------+--------+
|  ID|               Notes|   fname|
+----+--------------------+--------+
|2345|     Checked by John| Checked|
|2398|   Verified by Stacy|Verified|
|2328|Verified by Srini...|Verified|
|3983|Double Checked on...|      17|
+----+--------------------+--------+



In [ ]:
df.withColumn('fname', f.regexp_extract(df['Notes'], '([\w /]+) (by) ([a-zA-Z]+)', 1)).show()

+----+--------------------+--------------------+
|  ID|               Notes|               fname|
+----+--------------------+--------------------+
|2345|     Checked by John|             Checked|
|2398|   Verified by Stacy|            Verified|
|2328|Verified by Srini...|            Verified|
|3983|Double Checked on...|Double Checked on...|
+----+--------------------+--------------------+



### .join(대상 DataFrame, on= '기준 컬럼', how= '조인 방법')

In [ ]:
emp= [('홍길동',1),('이순신',2),
      ('임꺽정',3),('김철수',3),('김철수1',5)]
dept= [('개발',1), ('연구',2),
        ('영업',3),('기획',4)]
empA= sqlCtx.createDataFrame(emp, ['name','deptid'])
deptB= sqlCtx.createDataFrame(dept, ['deptname','deptid'])

In [ ]:
empA.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [ ]:
deptB.show()

+--------+------+
|deptname|deptid|
+--------+------+
|    개발|     1|
|    연구|     2|
|    영업|     3|
|    기획|     4|
+--------+------+



In [ ]:
empA.join(deptB, on='deptid', how='inner').show()

+------+------+--------+
|deptid|  name|deptname|
+------+------+--------+
|     1|홍길동|    개발|
|     3|임꺽정|    영업|
|     3|김철수|    영업|
|     2|이순신|    연구|
+------+------+--------+



In [ ]:
empA.join(deptB, on='deptid', how='left').show()

+------+-------+--------+
|deptid|   name|deptname|
+------+-------+--------+
|     5|김철수1|    null|
|     1| 홍길동|    개발|
|     3| 임꺽정|    영업|
|     3| 김철수|    영업|
|     2| 이순신|    연구|
+------+-------+--------+



In [ ]:
empA.join(deptB, on='deptid', how='right').show()

+------+------+--------+
|deptid|  name|deptname|
+------+------+--------+
|     1|홍길동|    개발|
|     3|임꺽정|    영업|
|     3|김철수|    영업|
|     2|이순신|    연구|
|     4|  null|    기획|
+------+------+--------+



In [ ]:
empA.join(deptB, on='deptid', how='full').show()

+------+-------+--------+
|deptid|   name|deptname|
+------+-------+--------+
|     5|김철수1|    null|
|     1| 홍길동|    개발|
|     3| 임꺽정|    영업|
|     3| 김철수|    영업|
|     2| 이순신|    연구|
|     4|   null|    기획|
+------+-------+--------+



# 퀴즈
```
spark데이터프레임을 만들고
inner, left, right, full join 을 테스트하시요
```

In [ ]:
# name, id
testA= [('A',1),('B',2),('C',3),('D',4)]
# name, myid
testB= [('E',1),('A',2),('C',3),('F',4)]

In [ ]:
testAA= sqlCtx.createDataFrame(testA, ['name','id'])
testBB= sqlCtx.createDataFrame(testB,['name', 'myid'])

In [ ]:
testAA.show()

+----+---+
|name| id|
+----+---+
|   A|  1|
|   B|  2|
|   C|  3|
|   D|  4|
+----+---+



In [ ]:
testBB.show()

+----+----+
|name|myid|
+----+----+
|   E|   1|
|   A|   2|
|   C|   3|
|   F|   4|
+----+----+



In [ ]:
testAA.join(testBB, how='inner', on='name').show()

+----+---+----+
|name| id|myid|
+----+---+----+
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



In [ ]:
testAA.join(testBB, how='left', on='name').show()

+----+---+----+
|name| id|myid|
+----+---+----+
|   B|  2|null|
|   D|  4|null|
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



In [ ]:
testAA.join(testBB, how='right', on='name').show()

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+



In [ ]:
testAA.join(testBB, how='full', on='name').show()

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   B|   2|null|
|   D|   4|null|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+



In [ ]:
jdf= testAA.join(testBB, how='full', on='name')
jdf.show()

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   B|   2|null|
|   D|   4|null|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+



In [ ]:
jdf= jdf.fillna({'id': 5})
jdf.show()

+----+---+----+
|name| id|myid|
+----+---+----+
|   F|  5|   4|
|   E|  5|   1|
|   B|  2|null|
|   D|  4|null|
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



### .dropna(subset= '컬럼')
- 해당 컬럼에 null값이 포함된 행을 삭제

In [ ]:
jdf.dropna(subset='myid').show()

+----+---+----+
|name| id|myid|
+----+---+----+
|   F|  5|   4|
|   E|  5|   1|
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



### .unionAll(대상 DataFrame)
- union과의 차이는 중복을 허용

In [ ]:
testAA.unionAll(testBB).show()

+----+---+
|name| id|
+----+---+
|   A|  1|
|   B|  2|
|   C|  3|
|   D|  4|
|   E|  1|
|   A|  2|
|   C|  3|
|   F|  4|
+----+---+



## 메모리 회수

In [ ]:
sc.stop()